In [1]:
    !pip install requests pandas


In [6]:
import requests
import pandas as pd

api_key = "5de77c31d99f9e83d1f01f4def19932bcd0ef44d"
base_url = "https://api.census.gov/data/{year}/acs/acs1"
years = [2021, 2022, 2023]
unemployment_data = []


In [7]:
for year in years:
    url = base_url.format(year=year)
    params = {
        "get": "NAME,B23025_001E,B23025_005E",  # State name, total labor force, and number of unemployed
        "for": "state:*",  # Get data for all states
        "key": api_key
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        unemployment_data.extend(data)
    else:
        print(f"Failed to retrieve data for {year}")
    


In [8]:
# Convert the data to a DataFrame
unemployment_df = pd.DataFrame(unemployment_data[1:], columns=unemployment_data[0])

# Rename the columns 
unemployment_df.rename(columns={
    "B23025_001E": "Total Labor Force",
    "B23025_005E": "Number of Unemployed"
}, inplace=True)

# Convert Labor Force and Number of Unemployed columns to numeric if necessary
unemployment_df["Total Labor Force"] = pd.to_numeric(unemployment_df["Total Labor Force"], errors="coerce")
unemployment_df["Number of Unemployed"] = pd.to_numeric(unemployment_df["Number of Unemployed"], errors="coerce")

# Calculate Unempl Rate
unemployment_df["Unemployment Rate"] = (unemployment_df["Number of Unemployed"] / unemployment_df["Total Labor Force"]) * 100


unemployment_df.head()


,NAME,Total Labor Force,Number of Unemployed,state,Unemployment Rate
0,Alabama,4046627.0,123808.0,01,3.059536
1,Puerto Rico,2794959.0,163354.0,72,5.844594
2,Arizona,5852913.0,203756.0,04,3.481275
3,Arkansas,2405035.0,77486.0,05,3.221824
4,California,31507237.0,1649320.0,06,5.234734


In [9]:
# Export to CSV
unemployment_df.to_csv("unemployment_rate_by_state_2021_2023.csv", index=False)

